### Постановка задачи
Рассмотрим несколько моделей линейной регрессии, чтобы выяснить более оптимальную для первых 20 зданий.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, BayesianRidge

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[(energy["building_id"]<20)]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
del buildings
del weather
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 10.39 Мб (минус 70.5 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           175680 non-null  datetime64[ns]
 1   building_id         175680 non-null  int8          
 2   meter_reading       175680 non-null  float16       
 3   primary_use         175680 non-null  category      
 4   air_temperature     175620 non-null  float16       
 5   cloud_coverage      99080 non-null   float16       
 6   dew_temperature     175620 non-null  float16       
 7   precip_depth_1_hr   175660 non-null  float16       
 8   sea_level_pressure  173980 non-null  float16       
 9   wind_direction      170680 non-null  float16       
 10  wind_speed          175680 non-null  float16       
dtypes: category(1), datetime64[ns](1), float16(8), int8(1)
memory usage: 4.4 MB
None


In [4]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

In [5]:
energy_train,energy_test = train_test_split(energy[energy['meter_reading'] > 0],test_size=0.2)

In [6]:
from sklearn.metrics import *

In [7]:
hours = range(0,24)
buildings = range(0,energy_train['building_id'].max() + 1)
lr_columns = ['meter_reading_log','hour','building_id','is_holiday']
for wday in range(0,7):
    lr_columns.append('is_wday' + str(wday))


Линейная регрессия
\begin{equation}
z = Ax + By + C, |z-z_0|^2 \rightarrow min
\end{equation}
Лассо + LARS Лассо
\begin{equation}
\frac{1}{2n}|z-z_0|^2 + a(|A|+|B|) \rightarrow min
\end{equation}
Гребневая регрессия
\begin{equation}
|z-z_0|^2 + a(A^2 + B^2) \rightarrow min
\end{equation}
ElasticNet: Лассо + Гребневая регрессия
\begin{equation}
\frac{1}{2n}|z-z_0|^2 + \alpha p|A^2+B^2| + (\alpha - p)(|A|+|B|)/2 \rightarrow min
\end{equation}

In [8]:
lr_models = {
    "LinearRegression":LinearRegression,
    "Lasso-0.01":Lasso,
    "Lasso-0.1":Lasso,
    "Lasso-1.0":Lasso,
    "Ridge-0.01":Ridge,
    "Ridge-0.1":Ridge,
    "Ridge-1.0":Ridge,
    "ELasticNet-1-1":ElasticNet,
    "ELasticNet-0.1-1":ElasticNet,
    "ELasticNet-1-0.1":ElasticNet,
    "ELasticNet-0.1-0.1":ElasticNet,
    "BayesianRidge":BayesianRidge
}
energy_train_lr = pd.DataFrame(energy_train,columns=lr_columns)

In [11]:
lr_models_scores = {}
for _ in lr_models:
    lr_model = lr_models[_]
    energy_lr_scores = [[]] * len(buildings)
    for building in buildings:
        energy_lr_scores[building] = [0] * len(hours)
        energy_train_b = energy_train_lr[energy_train_lr['building_id'] == building]
        for hour in hours:
            energy_train_bh = energy_train_b[energy_train_b['hour'] == hour]
            y = energy_train_bh['meter_reading_log']
            x = energy_train_bh.drop(['meter_reading_log','hour','building_id'],axis=1)
            if _ in ['Ridge-0.1','Lasso-0.1']:
                model = lr_model(alpha=0.1,fit_intercept=False).fit(x,y)
            elif _ in ['Ridge-0.01','Lasso-0.01']:
                model = lr_model(alpha=0.01,fit_intercept=False).fit(x,y)
            elif _ == 'ElasticNet-1-1':
                model = lr_model(alpha=1,l1_ratio=1,fit_intercept=False).fit(x,y)
            elif _ == 'ElasticNet-1-0.1':
                model = lr_model(alpha=1,l1_ratio=0.1,fit_intercept=False).fit(x,y)
            elif _ == 'ElasticNet-0.1-1':
                model = lr_model(alpha=0.1,l1_ratio=1,fit_intercept=False).fit(x,y)
            elif _ == 'ElasticNet-0.1-0.1':
                model = lr_model(alpha=0.1,l1_ratio=0.1,fit_intercept=False).fit(x,y)
            else:
                model = lr_model(fit_intercept=False).fit(x,y)
            energy_lr_scores[building][hour] = r2_score(y,model.predict(x))
    lr_models_scores[_] = np.mean(energy_lr_scores)
print(lr_models_scores)

{'LinearRegression': 0.13262979264188432, 'Lasso-0.01': -0.19060262906360775, 'Lasso-0.1': -31.177850789729533, 'Lasso-1.0': -2429.7711638954247, 'Ridge-0.01': 0.13221503036076057, 'Ridge-0.1': 0.09151042473014116, 'Ridge-1.0': -3.6576414865818228, 'ELasticNet-1-1': -2105.6896312858844, 'ELasticNet-0.1-1': -2105.6896312858844, 'ELasticNet-1-0.1': -2105.6896312858844, 'ELasticNet-0.1-0.1': -2105.6896312858844, 'BayesianRidge': 0.13261999641258232}


In [14]:
energy_lr = []
energy_ridge = []
energy_br = []
for building in buildings:
    energy_lr.append([])
    energy_ridge.append([])
    energy_br.append([])
    energy_train_b = energy_train_lr[energy_train_lr['building_id'] == building]
    for hour in hours:
        energy_lr[building].append([0] * (len(lr_columns)-3))
        energy_ridge[building].append([0] * (len(lr_columns)-3))
        energy_br[building].append([0] * (len(lr_columns)-3))
        energy_train_bh = energy_train_b[energy_train_b['hour'] == hour]
        y = energy_train_bh['meter_reading_log']
        if len(y) > 0:
            x = energy_train_bh.drop(['meter_reading_log','hour','building_id'],axis=1)
            model = LinearRegression(fit_intercept=False).fit(x,y)
            energy_lr[building][hour] = model.coef_
            model = Ridge(alpha=0.01,fit_intercept=False).fit(x,y)
            energy_ridge[building][hour] = model.coef_
            model = BayesianRidge(fit_intercept=False).fit(x,y)
            energy_br[building][hour] = model.coef_
print(energy_lr[0][0])
print(energy_ridge[0][0])
print(energy_br[0][0])

[-0.05204313  5.44504565  5.41921165  5.47881611  5.41753305  5.43838778
  5.45137392  5.44059806]
[-0.04938976  5.44244413  5.41674949  5.47670968  5.41516617  5.43591691
  5.44949479  5.43872264]
[-0.05138182  5.44439819  5.41859905  5.47829205  5.41694412  5.43777302
  5.45090643  5.44013149]
